<a href="https://colab.research.google.com/github/kevinbui0410/Coursera_Capstone/blob/main/Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Beautiful Soup to get data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M**

In [1]:
import requests
from bs4 import BeautifulSoup


url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/39.0.2171.95 Safari/537.36'}

page = requests.get(url,headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')



**Append data from wiki to pandas DataFrame**

In [18]:
import pandas as pd
import numpy as np

ds = []
columns = ['Postal Code', 'Borough', 'Neighbourhood']
tables = soup.find_all(class_='wikitable')
for table_neighbor in tables:
  rows = table_neighbor.find_all('tr')
  for row in rows:
    cells = row.find_all('td')
    if len(cells) == 3:
      if (cells[1].text.strip() != 'Not assigned') and (cells[2].text.strip() != 'Not assigned'):
        ds.append([cells[0].text.strip(),cells[1].text.strip(),cells[2].text.strip()])
      elif (cells[1].text.strip() != 'Not assigned') and (cells[2].text.strip() != 'Not assigned'):
        ds.append([cells[0].text.strip(),cells[1].text.strip(),cells[1].text.strip()])
ds_neighbor = pd.DataFrame(ds,columns=columns)
ds_neighbor.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
ds_neighbor.shape

(103, 3)